In [1]:
import numpy as np
import napari
import mrcfile
import urllib.request
import tifffile
import tempfile

In [2]:
def download_file_from_ftpurl(url):
    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(response.read())
        temp_file_path = temp_file.name

    return buffer

In [6]:
#Load data
mrc_buffer = download_fileio_from_ftpurl("https://ftp.ebi.ac.uk/empiar/world_availability/10988/data/VPP/tomograms/TS_0005.rec")
mrc_buffer.seek(0)


0

In [9]:
mrc_buffer

In [13]:
mrc_buffer.getbuffer().nbytes

445441024

In [14]:
with mrcfile.mmap(mrc_buffer, permissive=True) as f:
    data0=f.data.copy()

FileNotFoundError: [Errno 2] No such file or directory: '<_io.BytesIO object at 0x7f5448cc9310>'

In [ ]:
data0.shape

(500, 928, 960)

In [ ]:
data0.max()

127

In [ ]:
data0.min()

-128

Be careful with negative int values

Get respective cyto-ribossomes and membranes and join together 

In [6]:
with mrcfile.open("/ceph/users/ypu66991/data/Others/EMPIAR10988_Mahamid/data/VPP/labels/TS_0005_cyto_ribosomes.mrc") as f:
    labels_rib=f.data.copy().astype(np.uint8)

In [7]:
labels_rib.max()

1

In [8]:
labels_rib.shape

(500, 928, 960)

In [9]:
with mrcfile.open("/ceph/users/ypu66991/data/Others/EMPIAR10988_Mahamid/data/VPP/labels/TS_0005_membranes.mrc") as f:
    labels_memb=f.data.copy().astype(np.uint8)

In [10]:
labels_memb.max()

1

In [11]:
labels_memb.shape

(500, 928, 960)

join labels together, but be careful that there is some overlap. In overlap regions use ribos label

ribo=1
memb=2

In [12]:
labels_rib_memb = labels_rib+2*labels_memb


In [13]:
labels_rib_memb.max()

3

In [14]:
labels_rib_memb [labels_rib_memb==3] = 1 #replace overalp

ensure that voxels are not shared, otherwise we would have max2

In [15]:
labels_rib_memb.max()

2

check napari full volume

In [16]:
NV=napari.Viewer()
NV.add_image(data0)
NV.add_labels(labels_memb)
NV.add_labels(labels_rib)

<Labels layer 'labels_rib' at 0x7fd2002e7f70>

Crop centre 256 cube and save

In [17]:
c0 = [int(i/2) for i in data0.shape ]
c0

[250, 464, 480]

In [18]:
#Adjust centre
c0 = (269,345,390)

In [19]:
data0_crop = data0[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

In [20]:
labels_rib_memb_crop = labels_rib_memb[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

Check in napari

In [21]:
NV=napari.Viewer()
NV.add_image(data0_crop)
NV.add_labels(labels_rib_memb_crop)

<Labels layer 'labels_rib_memb_crop' at 0x7fd1d03fad00>

Save

In [22]:
tifffile.imwrite("TS_0005_crop.tif", data0_crop)
tifffile.imwrite("TS_0005_membranes_crop.tif", data0_crop)